Strong perturbation
===================

In this numerical experiment, one considers the propagation of a "strong" perturbation in a collisionless plasma model. The perturbation is initiated by sinusoidal city with an amplitude on the order of the ion acoustic velocity. The wavelength of this sinusoid is equal to the size of the simulation domain and moving in the $x$ direction.

The perturbation being strong, things will not occure in a linear way and we aim at understanding how thisvelocity fluctuation will develop.

In [ ]:
import os
import sys

In [ ]:
home = os.environ['HOME']
work_path = os.path.join(home, 'far/farTeach/2026/strong')
src_path = os.path.join(home, 'far/PHARE')
build_path = os.path.join(home, 'far/builds/release/ufunc')

In [ ]:
sys.path.append(os.path.join(src_path, "pyphare"))

In [ ]:
import subprocess
import pyphare
import matplotlib.pyplot as plt
from pyphare.pharesee.run import Run
from pyphare.core.operators import dot, cross, sqrt, modulus, grad
from pyphare.core.ufuncs import gF, peakIds
import numpy as np
from numpy import polyfit

## Run the `PHARE` code in dedicated conditions

In [ ]:
Ti = 0.005
Te = 0.005
run_name = "spa"

As treated in the weak perturbation" case, the initial velocity profile is in the $X$ direction...

In [ ]:
run_path = os.path.join(work_path, run_name)

In [ ]:
if os.path.isdir(run_path):
    files = os.listdir(run_path)
else:
    files = []

In [ ]:
if 'PYTHONPATH' in os.environ:
    os.environ['PYTHONPATH'] += os.pathsep + os.path.join(src_path, "pyphare")
else:
    os.environ['PYTHONPATH'] = os.pathsep + os.path.join(src_path, "pyphare")
os.environ['PYTHONPATH'] += os.pathsep + build_path

In [ ]:
if 'ions_charge_density.h5' not in files :
    os.chdir(work_path)
    # subprocess.run(['/usr/bin/python3', work_path+'/wp.py', run_name, str(Te), str(Ti)], env=os.environ)
    subprocess.run(['mpirun', '-n', '2', '/usr/bin/python3', work_path+'/sp.py', run_name, str(Te), str(Ti)], env=os.environ)

In [ ]:
run  = Run(run_path)

In [ ]:
times = np.arange(0, 21, 4)

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
fig, ax = plt.subplots(figsize=(12,2))

for i, time in enumerate(times):
    V = run.GetVi(time)
    v = gF(V, sigma=6)

    v.plot(qty='x', ax=ax, ls='solid', lw=2.0, color=colors[i%10], ylabel='X-velocity')

In [ ]:
fig, ax = plt.subplots(figsize=(12,2))

for i, time in enumerate(times):
    N = run.GetNi(time)
    n = gF(N, sigma=6)

    n.plot(qty='value', ax=ax, ls='solid', lw=2.0, color=colors[i%10], ylabel='X-velocity')

In [ ]:
time=20
r = Run(run_path)
ions = r.GetParticles(time, ["protons",])

fig, ax = plt.subplots(figsize=(10,4))

p,f = ions.dist_plot(axis=("x", "Vx"),
                     ax=ax,
                     norm = 0.4,
                     finest=True,
                     gaussian_filter_sigma=1,
                     vmin=-2,vmax=2,
                     dv=0.01,
                     title="weak perturbation at time : {:.2f}".format(time),
                    )

In [ ]:
plt.plot([1, 2, 3], [2, 3, 5], color='b', linestyle=None, marker='.', linewidth=0, markersize=0.5, alpha=1)

In [ ]:
a=np.arange(0,30,0.2)

In [ ]:
a[::2] # subsampling every 2 poionts

In [ ]:
import os
import sys
phare_root = os.path.expanduser("~/codes/far/PHARE/")
sys.path.append(os.path.join(phare_root, "pyphare"))
from pyphare.pharesee.run import Run

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter1d as gf
from numpy import polyfit
from scipy.signal import find_peaks
%matplotlib widget

In [ ]:
root_path = "/home/smets/FluKi/2024/run/strong"

In [ ]:
def get_velocities(r,times, sigma=6):
    for it,t in enumerate(times):
        Vi = r.GetVi(t, merged=True)
        Vix = Vi["Vx"][0]
        xV = Vi["Vx"][1][0]
        if sigma >0:
            v = gf(Vix(xV),sigma)
        else:
            v = Vix(xV)
        if it==0 :
            vt = np.zeros((len(v),len(times)))
        vt[:,it] = v
    return xV, vt

In [ ]:
def get_densities(r,times, sigma=6):
    for it,t in enumerate(times):
        N = r.GetNi(t, merged=True)
        Ni = N["rho"][0]
        x = N["rho"][1][0]
        if sigma >0:
            n = gf(Ni(x),sigma)
        else:
            n = Ni(x)
        if it==0 :
            nt = np.zeros((len(n),len(times)))
        nt[:,it] = n
    return x, nt

In [ ]:
def get_electric(r,times, sigma=6):
    for it,t in enumerate(times):
        E = r.GetE(t, merged=True)
        Ex = E["Ex"][0]
        x = E["Ex"][1][0]
        if sigma >0:
            e = gf(Ex(x),sigma)
        else:
            e = Ex(x)
        if it==0 :
            et = np.zeros((len(e),len(times)))
        et[:,it] = e
    return x, et

In [ ]:
def profiles(times, x, Vs, vmin=None,vmax=None, marker=None, **kw):
    fig, ax = plt.subplots()
    for it,t in enumerate(times):
        ax.plot(x, Vs[:,it], label=r"t={:5.2f}".format(t), marker=marker)
        ax.set_ylim((-2.5, 2.5))
        ax.axhline(0, ls='--', color='k')
        if vmin is not None and vmax is not None:
            ax.set_ylim((vmin,vmax))
    ax.legend(ncol=4)

In [ ]:
Te = 0.005
path = root_path+"/"+"sp1"

In [ ]:
times   = np.asarray((0, 2, 4, 6, 8, 10, 12, 14, 16))
r = Run(path)

In [ ]:
x, Vs = get_velocities(r,times, sigma=0)

In [ ]:
profiles(times, x, Vs)

In [ ]:
x, Ns   = get_densities(r,times, sigma=0)

In [ ]:
profiles(times, x, Ns, -0.2, 10)

In [ ]:
x, E   = get_electric(r,times, sigma=0)

In [ ]:
profiles(times, x, E, -0.01, 0.01)

In [ ]:
time=20
ions = r.GetParticles(time, ["protons",])
fig, (ax1,ax2) = plt.subplots(figsize=(10, 4), ncols=2)
p,f=ions.dist_plot(axis=("x", "Vx"),
                   ax=ax1,
                   norm=0.4,
                   finest=True,
                   # levels=(0,),  # default is (0,), tuple of all levels to overplot
                   # bulk=True,  # draw line(s) at mean vel. on vel. axis
                   # xlim=(0,20),  # default takes full data range
                   # ylim=(-2,2),  # default takes full data range
                   # sigma = (1,1),  # gaussian filter 2D
                   # filename=myImage.png,  # will save result in file
                   gaussian_filter_sigma=0,
                   vmin=-4, vmax=4,
                   dv=0.2,  # 0.05,
                   title="Strong perturbation at time : {:.2f}".format(time), # figure title
                   # xlabel="my x label", # default is name of chosen axis[0]
                   # ylabel= "my y label", # default is name of chosen axis[1]
                   # kde = True,     # default False, draws kernel density estimate contours
                   interp=True,
                  )

In [ ]:
xcuts = (62.0, 68.0)
for xcut in xcuts:
    ax1.axvline(xcut, ls="--")
    d,xbin,ybin= f[2:5]
    ax2.plot(ybin, d(xcut, ybin))

In [ ]:
import sys
sys.path = ["/home/aunai/PHARE/pyphare"] + sys.path

In [ ]:
import os
import sys
phare_root = os.path.expanduser("~/codes/far/PHARE/")
sys.path.append(os.path.join(phare_root, "pyphare"))

In [ ]:
from pyphare.pharesee.run import Run
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter1d as gf
from numpy import polyfit
from scipy.signal import find_peaks
import glob
%matplotlib widget

In [ ]:
def get_velocities(r,times, sigma=6):
    for it,t in enumerate(times):
        Vi = r.GetVi(t, merged=True)
        Vix = Vi["Vx"][0]
        xV = Vi["Vx"][1][0]
        if sigma >0:
            v = gf(Vix(xV),sigma)
        else:
            v = Vix(xV)
        if it==0 :
            vt = np.zeros((len(v),len(times)))
        vt[:,it] = v
    return xV, vt

def get_densities(r,times, sigma=6):
    for it,t in enumerate(times):
        N = r.GetNi(t, merged=True)
        Ni = N["rho"][0]
        x = N["rho"][1][0]
        if sigma >0:
            n = gf(Ni(x),sigma)
        else:
            n = Ni(x)
        if it==0 :
            nt = np.zeros((len(n),len(times)))
        nt[:,it] = n
    return x, nt

def get_electric(r,times, sigma=6):
    for it,t in enumerate(times):
        E = r.GetE(t, merged=True)
        Ex = E["Ex"][0]
        x = E["Ex"][1][0]
        if sigma >0:
            e = gf(Ex(x),sigma)
        else:
            e = Ex(x)
        if it==0 :
            et = np.zeros((len(e),len(times)))
        et[:,it] = e
    return x, et

def profiles(times, x, Vs, vmin=None,vmax=None, marker=None, **kw):
    fig, ax = plt.subplots()
    for it,t in enumerate(times):
        ax.plot(x, Vs[:,it], label=r"t={:5.2f}".format(t), marker=marker)
        ax.set_ylim((-2.5, 2.5))
        ax.axhline(0, ls='--', color='k')
        if vmin is not None and vmax is not None:
            ax.set_ylim((vmin,vmax))
    ax.legend(ncol=4)



# Strong perturbation

We do now address the case of a velocity perturbation of the order, or larger, with respect to the sound speed $c=\sqrt{\gamma T}$. The simulations are one dimensional, along the $x$ axis

In [ ]:
root_path = "/home/smets/FluKi/2024/run/strong"
# "/DATA/fluidecinetique/strong_perturbation/"

In [ ]:
paths = glob.glob(root_path+"/sp*")
print(paths)
paths = [os.path.join(root_path, "sp1")]
print(paths)

## With electron temperature Te=0.2

In [ ]:
irun = 0
r = Run(paths[irun])
times   = np.asarray((0,1,2,3,4,6,8))
x, Vs   = get_velocities(r,times, sigma=0)
profiles(times, x, Vs)

In [ ]:
irun=0
r = Run(paths[irun])
times   = np.asarray((0,2,4,6,8))
x, Ns   = get_densities(r,times, sigma=0)
profiles(times, x, Ns, -0.2, 7)

In [ ]:
irun=0
r = Run(paths[irun])
times   = np.asarray((0,2,4,6,8))
x, E   = get_electric(r,times, sigma=0)
profiles(times, x, E, -0.5, 0.5)

In [ ]:
t=1
ions = r.GetParticles(t, ["protons",])
fig, (ax1,ax2) = plt.subplots(figsize=(10,4), ncols=2)
p,f=ions.dist_plot(axis=("x", "Vx"), # default [Vx,Vy]
               ax=ax1,            # default creates the figure
               norm = 0.4,     # max value, min is 0
                finest=True,  # only consider finest particles possible
               # levels=(0,)    # default is (0,), tuple of all levels to overplot
               #bulk=True,        # draw line(s) at mean vel. on vel. axis
               #xlim=(0,20),     # default takes full data range
               #ylim=(-2,2),     # default takes full data range
               #sigma = (1,1),   # gaussian filter 2D
               #filename=myImage.png, # will save result in file
               gaussian_filter_sigma=0,
               vmin=-4,vmax=4,
               dv=0.05,
               title="Strong perturbation  t = {:.4f}".format(t), # figure title
               #xlabel="my x label", # default is name of chosen axis[0]
               #ylabel= "my y label", # default is name of chosen axis[1]
               # kde = True,     # default False, draws kernel density estimate contours               
              )

In [ ]:
xcuts = (8,10.35)
for xcut in xcuts:
    ax1.axvline(xcut,  ls="--")
    d,xbin,ybin= f[2:5]
    ax2.plot(ybin, d(xcut, ybin))

In [ ]:
t=2
ions = r.GetParticles(t, ["protons",])
fig, (ax1,ax2) = plt.subplots(figsize=(10,4), ncols=2)
p,f=ions.dist_plot(axis=("x", "Vx"), # default [Vx,Vy]
               ax=ax1,            # default creates the figure
               norm = 0.4,     # max value, min is 0
                finest=True,  # only consider finest particles possible
               # levels=(0,)    # default is (0,), tuple of all levels to overplot
               #bulk=True,        # draw line(s) at mean vel. on vel. axis
               #xlim=(0,20),     # default takes full data range
               #ylim=(-2,2),     # default takes full data range
               #sigma = (1,1),   # gaussian filter 2D
   ensity VA0 = B0/√mpn0μ0. It thus follows that
the distances are normalized to the ion inertial length δi0 = VA0/Ωci0ensity VA0 = B0/√mpn0μ0. It thus follows that
the distances are normalized to the ion inertial length δi0 = VA0/Ωci0            #filename=myImage.png, # will save result in file
               gaussian_filter_sigma=1,
               vmin=-4,vmax=4,
               dv=0.05,
               title="Strong perturbation  t = {:.4f}".format(t), # figure title
               #xlabel="my x label", # default is name of chosen axis[0]
               #ylabel= "my y label", # default is name of chosen axis[1]
               # kde = True,     # default False, draws kernel density estimate contours               
              )

In [ ]:
xcuts = (8,10.35)
for xcut in xcuts:
    ax1.axvline(xcut,  ls="--")
    d,xbin,ybin= f[2:5]
    ax2.plot(ybin, d(xcut, ybin))

In [ ]:
t=5
ions = r.GetParticles(t, ["protons",])
fig, (ax1,ax2) = plt.subplots(figsize=(10,4), ncols=2)
p,f=ions.dist_plot(axis=("x", "Vx"), # default [Vx,Vy]
               ax=ax1,            # default creates the figure
               norm = 0.4,     # max value, min is 0
                finest=True,  # only consider finest particles possible
               # levels=(0,)    # default is (0,), tuple of all levels to overplot
               #bulk=True,        # draw line(s) at mean vel. on vel. axis
               #xlim=(0,20),     # default takes full data range
               #ylim=(-2,2),     # default takes full data range
               #sigma = (1,1),   # gaussian filter 2D
               #filename=myImage.png, # will save result in file
               gaussian_filter_sigma=1,
               vmin=-4,vmax=4,
               dv=0.05,
               title="Strong perturbation  t = {:.4f}".format(t), # figure title
               #xlabel="my x label", # default is name of chosen axis[0]
               #ylabel= "my y label", # default is name of chosen axis[1]
               # kde = True,     # default False, draws kernel density estimate contours               
              )

In [ ]:
xcuts = (8,10.35)
for xcut in xcuts:
    ax1.axvline(xcut,  ls="--")
    d,xbin,ybin= f[2:5]
    ax2.plot(ybin, d(xcut, ybin))

## Without electron temperature Te=0

In [ ]:
irun = 1
r = Run(paths[irun])
times   = np.asarray((0,1,2,3,4,6,8))
x, Vs   = get_velocities(r,times, sigma=0)
profiles(times, x, Vs)

In [ ]:
irun=1
r = Run(paths[irun])
times   = np.asarray((0,2,4,6,8))
x, Ns   = get_densities(r,times, sigma=0)
profiles(times, x, Ns, -0.2, 7)

In [ ]:
irun=1
r = Run(paths[irun])
times   = np.asarray((0,2,4,6,8))
x, E   = get_electric(r,times, sigma=0)
profiles(times, x, E, -0.5, 0.5)

In [ ]:
t=1
ions = r.GetParticles(t, ["protons",])
fig, (ax1,ax2) = plt.subplots(figsize=(10,4), ncols=2)
p,f=ions.dist_plot(axis=("x", "Vx"), # default [Vx,Vy]
               ax=ax1,            # default creates the figure
               norm = 0.4,     # max value, min is 0
                finest=True,  # only consider finest particles possible
               # levels=(0,)    # default is (0,), tuple of all levels to overplot
               #bulk=True,        # draw line(s) at mean vel. on vel. axis
               #xlim=(0,20),     # default takes full data range
               #ylim=(-2,2),     # default takes full data range
               #sigma = (1,1),   # gaussian filter 2D
               #filename=myImage.png, # will save result in file
               gaussian_filter_sigma=1,
               vmin=-4,vmax=4,
               dv=0.05,
               title="Strong perturbation  t = {:.4f}".format(t), # figure title
               #xlabel="my x label", # default is name of chosen axis[0]
               #ylabel= "my y label", # default is name of chosen axis[1]
               # kde = True,     # default False, draws kernel density estimate contours               
              )

In [ ]:
xcuts = (8,10.35)
for xcut in xcuts:
    ax1.axvline(xcut,  ls="--")
    d,xbin,ybin= f[2:5]
    ax2.plot(ybin, d(xcut, ybin))

In [ ]:
t=2
ions = r.GetParticles(t, ["protons",])
fig, (ax1,ax2) = plt.subplots(figsize=(10,4), ncols=2)
p,f=ions.dist_plot(axis=("x", "Vx"), # default [Vx,Vy]
               ax=ax1,            # default creates the figure
               norm = 0.4,     # max value, min is 0
                finest=True,  # only consider finest particles possible
               # levels=(0,)    # default is (0,), tuple of all levels to overplot
               #bulk=True,        # draw line(s) at mean vel. on vel. axis
               #xlim=(0,20),     # default takes full data range
               #ylim=(-2,2),     # default takes full data range
               #sigma = (1,1),   # gaussian filter 2D
               #filename=myImage.png, # will save result in file
               gaussian_filter_sigma=1,
               vmin=-4,vmax=4,
               dv=0.05,
               title="Strong perturbation  t = {:.4f}".format(t), # figure title
               #xlabel="my x label", # default is name of chosen axis[0]
               #ylabel= "my y label", # default is name of chosen axis[1]
               # kde = True,     # default False, draws kernel density estimate contours               
              )

In [ ]:
xcuts = (8,10.35)
for xcut in xcuts:
    ax1.axvline(xcut,  ls="--")
    d,xbin,ybin= f[2:5]
    ax2.plot(ybin, d(xcut, ybin))

In [ ]:
t=5
ions = r.GetParticles(t, ["protons",])
fig, (ax1,ax2) = plt.subplots(figsize=(10,4), ncols=2)
p,f=ions.dist_plot(axis=("x", "Vx"), # default [Vx,Vy]
               ax=ax1,            # default creates the figure
               norm = 0.4,     # max value, min is 0
                finest=True,  # only consider finest particles possible
               # levels=(0,)    # default is (0,), tuple of all levels to overplot
               #bulk=True,        # draw line(s) at mean vel. on vel. axis
               #xlim=(0,20),     # default takes full data range
               #ylim=(-2,2),     # default takes full data range
               #sigma = (1,1),   # gaussian filter 2D
               #filename=myImage.png, # will save result in file
               gaussian_filter_sigma=1,
               vmin=-4,vmax=4,
               dv=0.05,
               title="Strong perturbation  t = {:.4f}".format(t), # figure title
               #xlabel="my x label", # default is name of chosen axis[0]
               #ylabel= "my y label", # default is name of chosen axis[1]
               # kde = True,     # default False, draws kernel density estimate contours               
              )

In [ ]:
xcuts = (8,10.35)
for xcut in xcuts:
    ax1.axvline(xcut,  ls="--")
    d,xbin,ybin= f[2:5]
    ax2.plot(ybin, d(xcut, ybin))

## profiles as a function of Te

In [ ]:
paths

In [ ]:
r = Run(paths[0])
times   = np.asarray((0,2,4,6,8))
x, Ns   = get_densities(r,times, sigma=0)
profiles(times, x, Ns, -0.2, 7)